<a href="https://colab.research.google.com/github/Adith-blusim/BP-/blob/main/1hrdatatraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/para_monitor_2023-12-14.csv")
df = df.reset_index(drop=True)
df = df.drop(['Unix Timestamp','RESP','ECG','SPO2'],axis=1)
rdf = pd.read_csv("data/BP.csv")
rdf['Timestamp'] = pd.to_datetime(rdf['Timestamp'], format='%Y-%m-%d %H:%M:%S')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')
rdf['Timestamp'] = rdf['Timestamp'].dt.strftime('%H:%M:%S')
df.set_index('Time', inplace=True)
rdf = rdf.groupby('Timestamp').agg(lambda x: x.tolist()).reset_index()
rdf.set_index('Timestamp', inplace=True)
df=df.resample('S').ffill()
df.index = df.index.strftime('%H:%M:%S')
ldf = pd.merge(rdf,df, left_index=True, right_index=True)
ldf[['Sistolic', 'Distolic']] = ldf[['Sistolic', 'Distolic']].replace('', np.nan)
ldf.dropna(how='any', inplace=True)
ldf['Sistolic'] = pd.to_numeric(ldf['Sistolic'], errors='coerce')
ldf['Distolic'] = pd.to_numeric(ldf['Distolic'], errors='coerce')
ldf = ldf.drop(ldf[ldf['Sistolic'].gt(140)].index)
ldf = ldf.drop(ldf[ldf['Distolic'].gt(100)].index)
ldf

In [ ]:
from scipy.interpolate import PchipInterpolator

def interpolate(arr):

    pchip_interpolator = PchipInterpolator(np.arange(len(arr)), arr)
    x_interp = np.linspace(0, len(arr) - 1, 123)
    y_interp = pchip_interpolator(x_interp)
    return y_interp.tolist()

df_interpolated = ldf[['Chan1','Chan2','Chan3','Chan4']].applymap(interpolate)
df_interpolated

In [ ]:
from PyEMD import EMD
from PyEMD import EEMD
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, hilbert
# Sample rate and desired cutoff frequencies (in Hz)
fs = 138.0
lowcut = 0.5
highcut = 6.0
def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
# Apply the Butterworth band-pass filter to the data
bdf = pd.DataFrame()
bdf['buttered_channel1'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Chan1']]
bdf['buttered_channel2'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Chan2']]
bdf['buttered_channel3'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Chan3']]
bdf['buttered_channel4'] = [butter_bandpass_filter(data, lowcut, highcut, fs, order=3) for data in df_interpolated['Chan4']]

eemd = EMD()
imf1c1 = []
imf1c2 = []
imf1c3 = []
imf1c4 = []

for i in bdf['buttered_channel1']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c1.append(i1)

for i in bdf['buttered_channel2']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c2.append(i1)

for i in bdf['buttered_channel3']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c3.append(i1)

for i in bdf['buttered_channel4']:
    time = np.arange(0, len(i)) / fs
    IMFs = eemd.emd(i, time)
    i1 = IMFs[0]
    imf1c4.append(i1)

ins_phase_c1 = []
ins_phase_c2 = []
ins_phase_c3 = []
ins_phase_c4 = []

for i in imf1c1:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c1.append(instantaneous_phase)

for i in imf1c2:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c2.append(instantaneous_phase)

for i in imf1c3:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c3.append(instantaneous_phase)

for i in imf1c4:
    analytic_signal = hilbert(i)
    instantaneous_phase = np.angle(analytic_signal)
    ins_phase_c4.append(instantaneous_phase)

In [ ]:
from sklearn.model_selection import train_test_split
merged_ipc = np.stack((ins_phase_c1,ins_phase_c2,ins_phase_c3,ins_phase_c4), axis=1)
merged_ipc = np.swapaxes(merged_ipc, 1, 2)
print(merged_ipc.shape)

X_train = merged_ipc
Y_train = ldf[['Sistolic', 'Distolic']].to_numpy().astype('float32')
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dense, BatchNormalization
tf.keras.backend.clear_session()

input_shape =  (123, 4)
model = Sequential()

#first Conv layer
model.add(Conv1D(filters=100, kernel_size=21, strides=1, input_shape=input_shape))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#second Conv layer
model.add(Conv1D(filters=200, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#third Conv layer
model.add(Conv1D(filters=300, kernel_size=5))
model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling1D(pool_size=2))

#GlobalAveragePooling1D layer
model.add(GlobalAveragePooling1D())
#Final dense layer
model.add(Dense(2, activation='linear'))

model.compile(loss=tf.keras.losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
model.summary()
history = model.fit(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test), batch_size=64)

In [ ]:
predictions = model.predict(X_test)
yf = pd.DataFrame(predictions, columns=['Systolic(predicted)', 'Diastolic(predicted)'])
yf[['Systolic(actual)', 'Diastolic(actual)']] = pd.DataFrame(Y_test, columns=['Systolic(actual)', 'Diastolic(actual)'])
yf

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# Plotting training and validation loss
plt.figure(figsize=(10, 6))

# Training loss
plt.plot(history.history['loss'], label='Training Loss', marker='o')

# Validation loss
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')

# Adding labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Performance')
plt.legend()

# Adding grid for better readability
plt.grid(True)

# Show plot
plt.show()

In [ ]:
sample_indices = np.arange(len(yf))
plt.figure(figsize=(70,10))
# Plotting line graph for output 1
plt.plot(sample_indices, yf['Systolic(actual)'], label='Systolic(actual)', linestyle='-', color='blue')
plt.plot(sample_indices, yf['Systolic(predicted)'], label='Systolic(predicted)', linestyle='--', color='cyan')

# Plotting line graph for output 2
plt.plot(sample_indices, yf['Diastolic(actual)'], label='Diastolic(actual)', linestyle='-', color='green')
plt.plot(sample_indices, yf['Diastolic(predicted)'], label='Diastolic(predicted)', linestyle='--', color='lime')

# Set labels and title
plt.xlabel('Sample Index')
plt.ylabel('Output Values')
plt.title('Comparison of Predicted and True Outputs')

# Show legend
plt.legend()

# Show the plot
plt.show()